In [105]:

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path='/content/drive/MyDrive/covid19/covid detection/Train'
val_path='/content/drive/MyDrive/covid19/covid detection/Val'

In [5]:
image_size=[299,299]

In [6]:
inception=InceptionV3(input_shape=image_size+[3],weights='imagenet',include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [7]:
# don't train existing weights
for layer in inception.layers:
  layer.trainable=False

In [10]:
folders=glob('/content/drive/MyDrive/covid19/covid detection/Train/*')
classes=len(folders)

In [11]:
#add some layers
x=Flatten()(inception.output)
dense_final=Dense(classes,activation='softmax')(x)



In [12]:
#model object

model=Model(inputs=inception.input,outputs=dense_final)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [15]:
# tell the model what cost and optimization method to use
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']    
)

In [16]:
# use imagedata generator for giving images to model
train_gen=ImageDataGenerator(train_path,rescale=1/255,
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              horizontal_flip = True)
val_gen=ImageDataGenerator(val_path,rescale=1/255)

In [18]:
training_set=train_gen.flow_from_directory(train_path,
                                           target_size=(299,299),
                                           batch_size=16,
                                           class_mode='categorical')
val_set=val_gen.flow_from_directory(val_path,
                                    target_size=(299,299),
                                    batch_size=16,
                                    class_mode='categorical' )

Found 282 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [19]:
#model training

r=model.fit_generator(training_set,
            validation_data=val_set,
            epochs=20)
model.save('/content/drive/MyDrive/covid19/covid detection/model_inception.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
18/18 [==============================] - 99s 5s/step - loss: 1.4159 - accuracy: 0.9113 - val_loss: 0.6775 - val_accuracy: 0.9750
Epoch 2/20
18/18 [==============================] - 15s 815ms/step - loss: 0.8439 - accuracy: 0.9752 - val_loss: 0.4441 - val_accuracy: 0.9500
Epoch 3/20
18/18 [==============================] - 15s 829ms/step - loss: 0.9264 - accuracy: 0.9681 - val_loss: 0.9965 - val_accuracy: 0.9500
Epoch 4/20
18/18 [==============================] - 16s 891ms/step - loss: 0.3424 - accuracy: 0.9787 - val_loss: 0.7638 - val_accuracy: 0.9750
Epoch 5/20
18/18 [==============================] - 15s 823ms/step - loss: 0.0070 - accuracy: 0.9965 - val_loss: 0.2393 - val_accuracy: 0.9875
Epoch 6/20
18/18 [==============================] - 15s 819ms/step - loss: 0.2832 - accuracy: 0.9894 - val_loss: 0.3145 - val_accuracy: 0.9875
Epoch 7/20
18/18 [==============================] - 16s 835ms/step - loss: 0.0332 - accuracy: 0.9965 - val_loss: 0.2312 - val_accuracy: 0.9875
Ep

In [104]:
pred=model.predict(val_set)
pred.shape

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


(80, 2)

In [57]:
## More testing

In [67]:
classes_out={0:'Normal',1:'Covid'}

In [75]:
## lets define a function which can return outputs and show image
def output(img_path):
  img=image.load_img(img_path,target_size=(299,299))
  img=image.img_to_array(img)
  img=img/255
  img=np.expand_dims(img,axis=0)
  img=preprocess_input(img)
  ## make predictions
  prediction=model.predict(img)
  prediction=np.argmax(prediction)

  if prediction==0:
    return 'Person is {}'.format(classes_out[prediction])
  elif prediction==1:
    return 'Person has {}'.format(classes_out[prediction])

In [76]:
output('/content/drive/MyDrive/covid19/covid detection/Test/COVID/143.jpg')

'Person has Covid'

In [77]:
output('/content/drive/MyDrive/covid19/covid detection/Val/NORMAL/165.jpg')

'Person has Covid'

In [99]:
import os
images=os.listdir('/content/drive/MyDrive/covid19/covid detection/Val/COVID')
images[0]

'163.jpg'

In [ ]:
#/content/drive/MyDrive/covid19/covid detection/Val/COVID

In [100]:
paths=[]
for i in range(len(images)):

  paths.append(os.path.join('/content/drive/MyDrive/covid19/covid detection/Val/COVID',images[i]))


In [101]:
paths[:4]

['/content/drive/MyDrive/covid19/covid detection/Val/COVID/163.jpg',
 '/content/drive/MyDrive/covid19/covid detection/Val/COVID/177.jpg',
 '/content/drive/MyDrive/covid19/covid detection/Val/COVID/194.jpg',
 '/content/drive/MyDrive/covid19/covid detection/Val/COVID/158.jpg']

In [102]:
res=[]
for path in paths:
  res.append(output(path))

In [103]:
res

['Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person is Normal',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid',
 'Person has Covid']